# 44. Batch heritability with GCTA and SLURM

In [ ]:
# submit_heritability_jobs.R

# Load necessary libraries
library(data.table)
library(stringr)
library(tools)
library(bsseq)

# Function to generate SbLURM script
generate_slurm_script <- function(args, tag_pt1, tag_pt2, partition, acct, mem_per_cpu = "4G",
                                  cluster_specific_parameters = TRUE, nodes = NULL,
                                  ntasks_per_node = NULL, time = "24:00:00",
                                  module_load_conda = FALSE, run_script_path) {
  cpus_per_task <- if (is.null(args$num_cores) || args$num_cores == "all") {
    "#SBATCH --exclusive\n"
  } else {
    paste0("#SBATCH --cpus-per-task=", args$num_cores, "\n")
  }

  # Always set mem_per_cpu flag, even in exclusive mode
  mem_allocation <- paste0("#SBATCH --mem-per-cpu=", mem_per_cpu, "\n")

  # Construct argument string with proper quoting
  args_string <- paste("--", names(args), "=", shQuote(as.character(args)), sep = "", collapse = " ")
  #args_string <- paste(args_string, " --tag_pt1=", shQuote(tag_pt1), sep = "")
  #args_string <- paste(args_string, " --tag_pt2=", shQuote(tag_pt2), sep = "")

  # Absolute path to run_heritability.R
  run_script_abs_path <- normalizePath(run_script_path, mustWork = FALSE)
  if (!file.exists(run_script_abs_path)) {
    stop(paste("run_heritability.R not found at path:", run_script_abs_path))
  }

  if(!dir.exists("slurm_logs")) dir.create("slurm_logs")

  slurm_script <- paste0(
    "#!/bin/bash\n",
    if (cluster_specific_parameters) paste0(
      "#SBATCH --partition=", partition, "\n"
      #"#SBATCH -A ", acct, "\n"
    ),
    if (!is.null(nodes) && !is.null(ntasks_per_node)) paste0(
      "#SBATCH --nodes=", nodes, "\n",
      "#SBATCH --ntasks-per-node=", ntasks_per_node, "\n"
    ),
    cpus_per_task,
    mem_allocation,
    "#SBATCH --output=slurm_logs/slurm_output_", tag_pt1, "-", tag_pt2, ".out\n",
    "#SBATCH --job-name=", tag_pt1, "-", tag_pt2, "\n",
    "#SBATCH --time=", time, "\n",
    if (module_load_conda) "module load conda\n",
    "echo 'Activating Conda environment.'\n",
    "module load conda\n",
    "conda activate mwas\n",
    "echo 'Executing Rscript with arguments: Rscript ", run_script_abs_path, " ", args_string, "'\n",
    "Rscript ", shQuote(run_script_abs_path), " ", args_string, "\n"
  )

  return(list(script = slurm_script, path = paste0("slurm_scripts/", tag_pt1, "-", tag_pt2, ".sh")))
}

# Cluster-specific parameters
cluster_specific_parameters <- TRUE
acct <- "jhu152"               # Adjust as needed
time <- "48:00:00"             # Adjust as needed
partition <- "shared"          # Adjust as needed
module_load_conda <- FALSE     # Set to TRUE if you need to load Conda module
nodes <- 1                     # Adjust as needed
ntasks_per_node <- 1           # Adjust as needed

# Overwrite flag (set by user)
overwrite <- FALSE

# Define absolute paths
project_dir <- normalizePath(getwd())  # Assuming the script is run from the project root
scripts_dir <- file.path(project_dir, "scripts")
slurm_scripts_dir <- file.path(project_dir, "slurm_scripts")
heritability_results_dir <- file.path(project_dir, "heritability_results")
run_script_path <- file.path("run_heritability.R")  # Adjust if located elsewhere

# Create necessary directories if they don't exist
dir.create(slurm_scripts_dir, showWarnings = FALSE)
dir.create(heritability_results_dir, showWarnings = FALSE)

# Load and preprocess matched_df
matched_df_path <- file.path(project_dir, "09.5-OUT_matched_SNP_meth_cov_chunked_JHPCE.csv")
if (!file.exists(matched_df_path)) {
  stop(paste("Matched data file not found at path:", matched_df_path))
}
matched_df <- fread(matched_df_path)
matched_df <- matched_df[order(matched_df$SNP_data), ]

# Update file paths as needed
matched_df$SNP_data <- gsub("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/",
                            "/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas/",
                            matched_df$SNP_data)
matched_df$methylation_data <- gsub("/expanse/lustre/projects/jhu152/naglemi/mwas/pheno/",
                                    "/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/",
                                    matched_df$methylation_data)
matched_df$cov_file <- gsub("/expanse/lustre/projects/jhu152/naglemi/mwas/full_covariates/",
                            "/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/",
                            matched_df$cov_file)
matched_df$modified_methylation_data <- gsub("/expanse/lustre/projects/jhu152/naglemi/","/dcs04/lieber/statsgen/mnagle/",
                                             matched_df$modified_methylation_data)

# Subset to Chr 6g
#chr_of_interest <- 6
#matched_df_chr6 <- matched_df[Chr == 6]
#matched_df_chr7 <- matched_df[Chr == 7]

#matched_df <- rbind(matched_df_chr6, matched_df_chr7)

matched_df <- matched_df[grepl("all", matched_df$methylation_data), ]

# For initial testing, select only the first five rows
#test_run <- TRUE
#if (test_run) {
#  matched_df_chr6 <- matched_df_chr6[1:100]
#}

# Initialize a data.table to store job submission logs
job_log <- data.table(
  row = integer(),
  tag_pt1 = character(),
  tag_pt2 = character(),
  script_path = character(),
  sbatch_output = character(),
  status = character()
)

# Loop over each row in matched_df_chr6 to submit jobs
for (i in 1:nrow(matched_df)) {

  # Extract variables from the current row
  row <- matched_df[i]
  snp_data_path <- row$SNP_data
  methylation_data_path <- row$methylation_data
  cov_file <- row$cov_file
  modified_methylation_data <- row$modified_methylation_data
  chr <- row$Chr
  chunk_start <- row$chunk_start
  chunk_end <- row$chunk_end
  region <- row$region  # Assuming 'region' corresponds to the desired naming

  # Prepare output directory for this job
  outdir <- file.path(heritability_results_dir, paste0("heritability_", region))
  dir.create(outdir, recursive = TRUE, showWarnings = FALSE)

  # Define constant arguments
  constant_args <- list(
    Chr = chr,
    SNP_data = snp_data_path,
    methylation_data = methylation_data_path,
    cov_file = cov_file,
    modified_methylation_data = modified_methylation_data,
    wind = "10000",  # Adjust as needed
    chunk_start = chunk_start,
    chunk_end = chunk_end,
    gwas = paste0(dirname(matched_df$SNP_data[i]), "/"),    # **IMPORTANT:** Adjust to your GWAS path
    gcta = "/dcs04/lieber/statsgen/shizhong/software/gcta/gcta-1.94.1-linux-kernel-3-x86_64/gcta-1.94.1",     # **IMPORTANT:** Adjust to your GCTA executable path
    num_cores = 1                # Adjust if needed
    # outdir = outdir  # **REMOVE THIS LINE**
  )

  # Generate unique tags for the job
  snp_base <- file_path_sans_ext(basename(snp_data_path))
  meth_base <- file_path_sans_ext(basename(methylation_data_path))
  datetime_str <- format(Sys.time(), "%Y%m%d-%H%M%S")
  tag_pt1 <- paste(snp_base, meth_base, "herit", sep = "-")
  tag_pt2 <- datetime_str

  # Set the tags in constant_args
  constant_args$tag_pt1 <- tag_pt1
  constant_args$tag_pt2 <- tag_pt2

  # Define output file path
  output_file <- file.path(outdir, paste0(tag_pt1, "-", tag_pt2, "_heritability_results.RDS"))

  # Check if output file already exists
  if (!overwrite && file.exists(output_file)) {
    message("File ", output_file, " already exists. Skipping job ", i, "...")
    next
  }

  # Initialize error_message
  error_message <- NULL

  # Generate SLURM script
  script_info <- tryCatch({
    generate_slurm_script(
      args = constant_args,
      tag_pt1 = tag_pt1,
      tag_pt2 = tag_pt2,
      partition = partition,
      acct = acct,
      mem_per_cpu = "2G",
      cluster_specific_parameters = cluster_specific_parameters,
      nodes = nodes,
      ntasks_per_node = ntasks_per_node,
      time = time,
      module_load_conda = module_load_conda,
      run_script_path = run_script_path
    )
  }, error = function(e) {
    warning(paste("Error generating SLURM script for job", i, ":", e$message))
    error_message <<- e$message
    return(NULL)
  })

  # If script generation failed, log and skip
  if (is.null(script_info)) {
    job_log <- rbind(job_log, data.table(
      row = i,
      tag_pt1 = tag_pt1,
      tag_pt2 = tag_pt2,
      script_path = NA,
      sbatch_output = paste("Error generating script:", error_message),
      status = "Failed"
    ))
    next
  }

  # Write the SLURM script to a file
  writeLines(script_info$script, script_info$path)

  # Submit the job using sbatch and capture the output
  sbatch_result <- system2("sbatch", args = script_info$path, stdout = TRUE, stderr = TRUE)

  # Determine the status based on sbatch output
  if (grepl("^Submitted batch job", sbatch_result)) {
    status <- "Submitted"
  } else {
    status <- "Failed"
  }

  # Log the job submission details
  job_log <- rbind(job_log, data.table(
    row = i,
    tag_pt1 = tag_pt1,
    tag_pt2 = tag_pt2,
    script_path = script_info$path,
    sbatch_output = sbatch_result,
    status = status
  ))

  # Inform the user
  message(paste0("Submitted job ", i, " with tag ", tag_pt1, "-", tag_pt2, ". Status: ", status))

  # If submission failed, skip the sleep
  if (status == "Failed") {
    next
  }

  # Implement job submission limits and intervals if necessary
  sleeptime <- 31
  message(paste0("Sleeping for ", sleeptime, " seconds to avoid overloading the scheduler."))
  Sys.sleep(sleeptime)
  cat("\n")
}

# Save the job submission log
job_log_path <- file.path(slurm_scripts_dir, "job_submission_log.csv")
fwrite(job_log, file = job_log_path)

message("Job submission for Chr ", chr_of_interest, " completed. Check '", job_log_path, "' for details.")

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:data.table’:

    first, second


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Submitted job 2 with tag libd_chr1-chr1_all-herit-20241104-124852. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



Submitted job 3 with tag libd_chr1-chr1_all-herit-20241104-124923. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



Submitted job 4 with tag libd_chr1-chr1_all-herit-20241104-124954. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



Submitted job 5 with tag libd_chr1-chr1_all-herit-20241104-125025. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



...

Submitted job 461 with tag libd_chr10-chr10_all-herit-20241104-164732. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



Submitted job 462 with tag libd_chr10-chr10_all-herit-20241104-164804. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



Submitted job 463 with tag libd_chr10-chr10_all-herit-20241104-164835. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



Submitted job 464 with tag libd_chr10-chr10_all-herit-20241104-164906. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



Submitted job 465 with tag libd_chr10-chr10_all-herit-20241104-164938. Status: Submitted

Sleeping for 31 seconds to avoid overloading the scheduler.



## Debug

In [ ]:
# Debugging Script for Heritability Estimation
# Run this script in a Jupyter Notebook to debug the heritability estimation process.

# Load necessary libraries
library(data.table)
library(stringr)
library(bsseq)
library(tools)

# Define helper function to check file existence and print messages
check_and_report <- function(file_path, description) {
  if (file.exists(file_path)) {
    full_path <- normalizePath(file_path)
    current_wd <- getwd()
    # Since all paths are absolute, relative_path is same as full_path
    relative_path <- full_path
    message(paste0(description, " saved to full path ", full_path,
                   " from current wd ", current_wd,
                   " using relative path ", relative_path))
  } else {
    message(paste0("Failed to save ", description, " to ", file_path))
  }
}

# ---------------------------
# Step 1: Setup Environment
# ---------------------------

# Define absolute paths (Adjust these paths according to your environment)
project_dir <- normalizePath(getwd())  # Assuming the notebook is run from the project root
scripts_dir <- file.path(project_dir, "scripts")
heritability_results_dir <- file.path(project_dir, "heritability_results")
all_gcta_outputs_dir <- file.path(project_dir, "all_gcta_outputs")
run_script_path <- file.path("run_heritability.R")  # Adjust if located elsewhere

# Create necessary directories if they don't exist
dir.create(all_gcta_outputs_dir, showWarnings = FALSE, recursive = TRUE)
dir.create(heritability_results_dir, showWarnings = FALSE, recursive = TRUE)

# ---------------------------
# Step 2: Load and Preprocess matched_df
# ---------------------------

# Load matched_df
matched_df_path <- file.path(project_dir, "09.5-OUT_matched_SNP_meth_cov_chunked_JHPCE.csv")
if (!file.exists(matched_df_path)) {
  stop(paste("Matched data file not found at path:", matched_df_path))
}
matched_df <- fread(matched_df_path)
matched_df <- matched_df[order(matched_df$SNP_data), ]

# Update file paths as needed
matched_df$SNP_data <- gsub("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/",
                            "/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas/",
                            matched_df$SNP_data)
matched_df$methylation_data <- gsub("/expanse/lustre/projects/jhu152/naglemi/mwas/pheno/",
                                    "/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/",
                                    matched_df$methylation_data)
matched_df$cov_file <- gsub("/expanse/lustre/projects/jhu152/naglemi/mwas/full_covariates/",
                            "/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/",
                            matched_df$cov_file)
matched_df$modified_methylation_data <- gsub("/expanse/lustre/projects/jhu152/naglemi/mwas/",
                                             "/dcs04/lieber/statsgen/mnagle/mwas/",
                                             matched_df$modified_methylation_data)

# ---------------------------
# Step 3: Select a Single Job for Debugging
# ---------------------------

# Subset to Chr of interest (e.g., Chr 8 for debugging)
chr_of_interest <- 8
matched_df_chr <- matched_df[Chr == chr_of_interest]

# Check if there are any rows after subsetting
if (nrow(matched_df_chr) == 0) {
  stop(paste("No data found for chromosome", chr_of_interest))
}

# Select the first row for debugging
debug_row <- matched_df_chr[1]

# Extract variables from the first row
snp_data_path <- debug_row$SNP_data
methylation_data_path <- debug_row$methylation_data
cov_file <- debug_row$cov_file
modified_methylation_data <- debug_row$modified_methylation_data
chr <- debug_row$Chr
chunk_start <- debug_row$chunk_start
chunk_end <- debug_row$chunk_end
region <- debug_row$region  # Assuming 'region' corresponds to the desired naming

# ---------------------------
# Step 4: Prepare Arguments
# ---------------------------

# Define constant arguments as per master script
constant_args <- list(
  Chr = chr,
  SNP_data = snp_data_path,
  methylation_data = methylation_data_path,
  cov_file = cov_file,
  modified_methylation_data = modified_methylation_data,
  wind = "10000",  # Adjust as needed
  chunk_start = chunk_start,
  chunk_end = chunk_end,
  gwas = paste0(dirname(matched_df_chr$SNP_data[1]), "/"),    # Adjust to your GWAS path
  gcta = "/dcs04/lieber/statsgen/shizhong/software/gcta/gcta-1.94.1-linux-kernel-3-x86_64/gcta-1.94.1",     # Adjust to your GCTA executable path
  num_cores = 1                # Adjust if needed
  # outdir = outdir  # **REMOVE THIS LINE**
)

# Generate unique tags for the job
snp_base <- file_path_sans_ext(basename(snp_data_path))
meth_base <- file_path_sans_ext(basename(methylation_data_path))
datetime_str <- format(Sys.time(), "%Y%m%d-%H%M%S")
tag_pt1 <- paste(snp_base, meth_base, "herit", sep = "-")
tag_pt2 <- datetime_str

# Add tags to constant_args
constant_args$tag_pt1 <- tag_pt1
constant_args$tag_pt2 <- tag_pt2

# Define output directory for this job
outdir <- file.path(all_gcta_outputs_dir, paste0("gcta_output_", tag_pt1, "_", tag_pt2))
outdir <- gsub("\\+", "", outdir)  # Remove any '+' signs
dir.create(outdir, recursive = TRUE, showWarnings = FALSE)

# Display job tags and output directory
cat("Job Tags:\n")
cat("Tag Part 1:", tag_pt1, "\n")
cat("Tag Part 2:", tag_pt2, "\n")
cat("Output Directory:", outdir, "\n")

# ---------------------------
# Step 5: Verify Input Files
# ---------------------------

# Define input files to check
input_files_to_check <- list(
  "SNP data" = snp_data_path,
  "Methylation data" = methylation_data_path,
  "Covariate file" = cov_file,
  "Modified methylation data" = modified_methylation_data
)

# Check existence of input files
for (desc in names(input_files_to_check)) {
  file_path <- input_files_to_check[[desc]]
  if (!file.exists(file_path)) {
    stop(paste0(desc, " file not found at path: ", file_path))
  } else {
    check_and_report(file_path, desc)
  }
}

# ---------------------------
# Step 6: Load Methylation Data
# ---------------------------

p <- readRDS(modified_methylation_data)
check_and_report(modified_methylation_data, "Modified methylation data")

CpG_positions <- p@methylations_positions
p <- p@methylations

ind <- fread(cov_file)$ID  # Adjust if necessary
check_and_report(cov_file, "Covariate file")

# Create 'id' file in 'outdir' based on 'ind'
# Since fam has #IID and SEX, and PLINK2 expects two columns (FID and IID), set FID = IID
id_file <- file.path(outdir, "id")
write.table(data.frame(FID = 0, IID = ind), file = id_file, row.names = FALSE, col.names = FALSE, quote = FALSE)
check_and_report(id_file, "ID file")

# Initialize result list and index to avoid copying data when appending
res <- list()
res_index <- 1  # Index to track the position in the result list

# ---------------------------
# Step 11: Debugging - Select Specific CpG and Window
# ---------------------------

# For debugging, select a specific CpG index and window
debug_CpG_idx <- 1  # Adjust as needed
debug_wind_idx <- 1  # Adjust as needed

# Extract the selected CpG position
CpG_pos <- CpG_positions[debug_CpG_idx]
p1 <- ifelse(CpG_pos - as.numeric(constant_args$wind) > 0, CpG_pos - as.numeric(constant_args$wind), 0)
p2 <- CpG_pos + as.numeric(constant_args$wind)
gwas_prefix <- paste0(constant_args$gwas, "libd_chr", chr)

# Define PLINK2 output prefix
plink_out_prefix <- file.path(outdir, paste0("temp_", debug_CpG_idx, "_", debug_wind_idx))

# Display debugging information
cat("Debugging CpG Index:", debug_CpG_idx, "\n")
cat("CpG Position:", CpG_pos, "\n")
cat("Window Size:", constant_args$wind, "\n")
cat("Genomic Range:", p1, "-", p2, "\n")
cat("Number of Samples to Keep:", length(ind), "\n")
cat("Sample IDs (first 5):", paste(head(ind, 5), collapse = ", "), "\n")

# ---------------------------
# Step 12: Generate PLINK2 BED/BIM/FAM Files
# ---------------------------

# Construct PLINK2 command
command_plink <- paste(
  "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/plink2",
  "--pfile", shQuote(gwas_prefix),
  #"--silent",
  "--keep", shQuote(id_file),
  "--chr", chr,
  "--from-bp", p1,
  "--to-bp", p2,
  "--snps-only", "just-acgt",
  "--make-bed",
  "--threads", 1,
  "--out", shQuote(plink_out_prefix)
)
cat("Executing PLINK2 Command:\n")
cat(command_plink, "\n")

# Execute the PLINK2 command
system_ret <- system2(command_plink[1], args = command_plink[-1], stdout = TRUE, stderr = TRUE)

# Print the system call results
cat("Output and Errors from PLINK2:\n")
print(system_ret)

# Check if PLINK2 executed successfully
if (!file.exists(paste0(plink_out_prefix, ".bed"))) {
  cat("PLINK2 command failed or no output generated for CpG index ", debug_CpG_idx, ".\n")
  # Output PLINK2 log content for debugging
  plink_log_file <- paste0(plink_out_prefix, ".log")
  if (file.exists(plink_log_file)) {
    cat("PLINK2 Log Content:\n")
    plink_log <- readLines(plink_log_file)
    cat(paste(plink_log, collapse = "\n"), "\n")
  }
  stop("PLINK2 failed. Check the log for details.")
} else {
  cat("PLINK2 command executed successfully.\n")
}

# Check for PLINK2 output files
bed_file <- paste0(plink_out_prefix, ".bed")
bim_file <- paste0(plink_out_prefix, ".bim")
fam_file_out <- paste0(plink_out_prefix, ".fam")

check_and_report(bed_file, "PLINK BED file")
check_and_report(bim_file, "PLINK BIM file")
check_and_report(fam_file_out, "PLINK FAM file")

# ---------------------------
# Step 13: Create Phenotype File
# ---------------------------

pheno <- data.frame(FID = fam_data$FID[fam_data$IID %in% ind], IID = ind, Phenotype = p[, debug_CpG_idx])
pheno_path <- file.path(outdir, paste0("pheno_", debug_CpG_idx, "_", debug_wind_idx, ".txt"))
write.table(pheno, pheno_path, col.names = FALSE, row.names = FALSE, quote = FALSE, sep = "\t")
check_and_report(pheno_path, "Phenotype file")

# ---------------------------
# Step 14: Generate Genetic Relationship Matrix (GRM) with GCTA
# ---------------------------

command_grm <- paste(
  shQuote(constant_args$gcta),
  "--bfile", shQuote(plink_out_prefix),
  "--make-grm-bin",
  "--thread-num", 1,
  "--out", shQuote(plink_out_prefix)
)
cat("Executing GCTA GRM Command:\n")
cat(command_grm, "\n")

# Execute GCTA GRM command
system_ret_grm <- system(command_grm, intern = TRUE, ignore.stderr = FALSE)

# Check if GRM was created successfully
grm_bin <- paste0(plink_out_prefix, ".grm.bin")
grm_n <- paste0(plink_out_prefix, ".grm.N.bin")
grm_id <- paste0(plink_out_prefix, ".grm.id")

if (!file.exists(grm_bin) || !file.exists(grm_n) || !file.exists(grm_id)) {
  cat("GCTA GRM command failed or output files not generated.\n")
  stop("GCTA GRM failed. Check the log for details.")
} else {
  cat("GCTA GRM command executed successfully.\n")
}

# Check for GRM output files
check_and_report(grm_bin, "GRM BIN file")
check_and_report(grm_n, "GRM N BIN file")
check_and_report(grm_id, "GRM ID file")

# ---------------------------
# Step 15: Perform Heritability Estimation Using GCTA REML
# ---------------------------

# Define qcovar and covar file paths
qcovar_file <- sub("\\.csv$", ".qcovar", cov_file)
covar_file <- sub("\\.csv$", ".covar", cov_file)

# Construct GCTA REML command
command_reml <- paste(
  shQuote(constant_args$gcta),
  "--reml",
  "--grm-bin", shQuote(plink_out_prefix),
  "--pheno", shQuote(pheno_path),
  "--mpheno", 1,
  "--qcovar", shQuote(qcovar_file),
  "--covar", shQuote(covar_file),
  "--thread-num", 1,
  "--out", shQuote(plink_out_prefix)
)
cat("Executing GCTA REML Command:\n")
cat(command_reml, "\n")

# Execute GCTA REML command
system_ret_reml <- system(command_reml, intern = TRUE, ignore.stderr = FALSE)

# Check if REML was successful
hsq_file <- paste0(plink_out_prefix, ".hsq")
if (!file.exists(hsq_file)) {
  cat("GCTA REML command failed or .hsq file not generated.\n")
  # Output GCTA REML log content for debugging
  gcta_log_file <- paste0(plink_out_prefix, ".log")
  if (file.exists(gcta_log_file)) {
    cat("GCTA REML Log Content:\n")
    gcta_log <- readLines(gcta_log_file)
    cat(paste(gcta_log, collapse = "\n"), "\n")
  }
  stop("GCTA REML failed. Check the log for details.")
} else {
  cat("GCTA REML command executed successfully.\n")
}

# Check for REML output file
check_and_report(hsq_file, "REML HSQ file")

# ---------------------------
# Step 16: Read and Save Heritability Results
# ---------------------------

# Read the .hsq file
hsq_data <- fread(hsq_file, header = TRUE, fill = TRUE)

# Extract relevant values
temp2 <- list(
  V_G = hsq_data[1, 2],
  SE_V_G = hsq_data[1, 3],
  V_e = hsq_data[2, 2],
  SE_V_e = hsq_data[2, 3],
  Vp = hsq_data[3, 2],
  SE_Vp = hsq_data[3, 3],
  V_G_Vp = hsq_data[4, 2],
  SE_V_G_Vp = hsq_data[4, 3],
  logL = hsq_data[5, 2],
  logL0 = hsq_data[6, 2],
  LRT = hsq_data[7, 2],
  df = hsq_data[8, 2],
  Pval = hsq_data[9, 2],
  n = hsq_data[10, 2],
  site = paste0("chr", chr, "_", CpG_positions[debug_CpG_idx]),
  wind = constant_args$wind
)

# Validate results
if (is.null(temp2) || temp2$n <= 1) {
  message("Invalid result for CpG index ", debug_CpG_idx, ", window ", constant_args$wind, ". Skipping.")
} else {
  # Create a data.table for results
  res_dt <- data.table(temp2)
  
  # Define output file path inside 'outdir' and add '.csv' extension
  output_file <- file.path(outdir, paste0(file_path_sans_ext(basename(modified_methylation_data)), "_heritability_results_debug.csv"))
  
  # Save results as CSV
  fwrite(res_dt, file = output_file)
  check_and_report(output_file, "Final heritability results CSV")
  
  message(paste0("Heritability estimation completed successfully. Results saved to ", output_file))
}

In [ ]:
ret_code_plink <- system(command_plink, intern = TRUE)

In [ ]:
ret_code_plink

In [ ]:
# Define the command with all its arguments including the output directory for the log
plink_command <- paste(
  "plink2",
  "--pfile '/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas/libd_chr8'",
  "--keep '/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/all_gcta_outputs/gcta_output_libd_chr8-chr8_AA-herit_20241011-175524/id'",
  "--chr 8",
  "--from-bp 186751",
  "--to-bp 206751",
  "--snps-only just-acgt",
  "--make-bed",
  "--threads 1",
  "--out '/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/all_gcta_outputs/gcta_output_libd_chr8-chr8_AA-herit_20241011-175524/temp_1_1'",
  sep = " "
)

# Execute the command
system(plink_command, intern = FALSE) # intern = FALSE to let it run without capturing output in R

# Read and print the log file
log_file <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/all_gcta_outputs/gcta_output_libd_chr8-chr8_AA-herit_20241011-175524/temp_1_1.log"
if (file.exists(log_file)) {
  log_contents <- readLines(log_file)
  cat("Log File Contents:\n", paste(log_contents, collapse = "\n"))
} else {
  cat("Log file not found.\n")
}


In [ ]:
# Set up command and arguments
command <- "plink2"
args <- c(
  "--pfile", "/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas/libd_chr8",
  "--keep", "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/all_gcta_outputs/gcta_output_libd_chr8-chr8_AA-herit_20241011-175524/id",
  "--chr", "8",
  "--from-bp", "186751",
  "--to-bp", "206751",
  "--snps-only", "just-acgt",
  "--make-bed",
  "--threads", "1",
  "--out", "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/all_gcta_outputs/gcta_output_libd_chr8-chr8_AA-herit_20241011-175524/temp_1_1"
)

# Execute command
system2(command, args = args, stdout = FALSE, stderr = FALSE)

# Read and print the log file
log_file <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/all_gcta_outputs/gcta_output_libd_chr8-chr8_AA-herit_20241011-175524/temp_1_1.log"
if (file.exists(log_file)) {
  log_contents <- readLines(log_file)
  cat("Log File Contents:\n", paste(log_contents, collapse = "\n"))
} else {
  cat("Log file not found.\n")
}


In [ ]:
# ---------------------------
# Step 17: Cleanup Temporary Files (Optional for Debugging)
# ---------------------------

# Remove temporary PLINK files
temp_files_pattern <- paste0("^temp_", debug_CpG_idx, "_", debug_wind_idx)
temp_files <- list.files(outdir, pattern = temp_files_pattern, full.names = TRUE)
if (length(temp_files) > 0) {
  file.remove(temp_files)
  message(paste0("Temporary files removed: ", paste(basename(temp_files), collapse = ", ")))
}

# Remove phenotype file
if (file.exists(pheno_path)) {
  file.remove(pheno_path)
  message(paste0("Phenotype file removed: ", basename(pheno_path)))
}

# Remove GRM files
grm_files <- c(grm_bin, grm_n, grm_id)
existing_grm_files <- grm_files[file.exists(grm_files)]
if (length(existing_grm_files) > 0) {
  file.remove(existing_grm_files)
  message("GRM files removed: ", paste(basename(existing_grm_files), collapse = ", "))
}

# Remove HSQ file
if (file.exists(hsq_file)) {
  file.remove(hsq_file)
  message("HSQ file removed: ", basename(hsq_file))
}

cat("Debugging run completed.\n")

fam_file